In [5]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("LANGCHAIN_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "Self Improvement V2"

# Tools

In [6]:
from utils.tools import construct_tools, get_tools_descriptions
from langgraph.prebuilt import ToolNode

tools = construct_tools()
tools_descriptions = get_tools_descriptions(tools)
tool_node = ToolNode(tools)
from langchain_openai import ChatOpenAI

# Define State

In [8]:
from typing import Literal
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

MAX_ITERATIONS = 4

class State(TypedDict):
    input: str
    prediction: str
    critiques: Annotated[list[str], add]
    react_messages: Annotated[list, add]
    predictions: Annotated[list[str], add]
    iteration: Annotated[int, add]

from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage,HumanMessage

critic_prompt = ("Inspect the previous messages and identify any potential issues or errors. "
                 "Reflect on the process of problem-solving. "
                 "What's the problem of the previous answer? "
                 "If there are tool calls, what are the issues with tool calls? (Tool selection, parameters, etc.)"
                 "Is the final answer truthful?"
                 "Your response should be as brief as possible . ")

async def criticize(state):
    if state["react_messages"]:
        react_messages = state["react_messages"][-1]
        critique = await llm.ainvoke(react_messages + [HumanMessage(content=critic_prompt)])
        return {
            "critiques": [critique.content],
            "iteration": 1 
        }
    else:
        return {
            "critiques": ["An error occurred. Please try again."],
            "iteration": 1 
        }

react_prompt = ("Always remember that there may be multiple tools that can be used to complete a step! "
                "So if there are more than one tool that can be used, your response should contain multiple tools and their parameters in one message! "
                "When you are going to respond the final answer, you should follow the following format:\n\n"
                "FINAL ANSWER: <Your final answer>\n\n"
                "Always remember the final answer should be as concise as possible. (A single number, a phrase, etc. Not a sentence!)\n\n")


reacter = create_react_agent(model=llm, tools=tools, state_modifier=SystemMessage(content=react_prompt))

async def react(state):
    try:
        if state["critiques"]:
            critique = state["critiques"][-1]
            inputs = {"messages": state["react_messages"][-1] + [HumanMessage(content=critique), HumanMessage(content=f"Question: {state["input"]}")]}
            result = await reacter.ainvoke(input=inputs, config={'recursion_limit': 18})
        else:
            inputs = {"messages": [("user", state["input"])]}
            result = await reacter.ainvoke(input=inputs)
    except Exception as e:
        return {
            "predictions": ["None"]
        }
    return {
        "react_messages": [result["messages"][len(state["react_messages"][-1])+1:]],
        "predictions": [result["messages"][-1].content],
    }


# Either agent can decide to end
from typing import Literal

def should_end(state) -> Literal["critic", "__end__"]:
    if state["iteration"] >= MAX_ITERATIONS or len(state["predictions"]) > 1 and state["predictions"][-1].split("FINAL ANSWER:")[-1].strip() == state["predictions"][-2].split("FINAL ANSWER:")[-1].strip():
        return "__end__"
    else:
        return "critic"
    

builder = StateGraph(State)

builder.add_node("critic", criticize)
builder.add_node("react", react)

builder.add_edge(START, "critic")
builder.add_edge("critic", "react")

builder.add_conditional_edges("react", should_end)

graph = builder.compile()

In [15]:
import json
import asyncio


async def process(item, dataset_name:str="hotpot_qa", timeout: int = 300):
    if dataset_name in ["hotpot_qa", "trivia_qa", "ambig_qa"]:
        messages = [HumanMessage(content=f"{item['question']}\n{item['prediction']}")]
        input = {"react_messages": [messages], "input": item["question"], "prediction": item["prediction"], "iteration": 0}
    elif dataset_name in ["gsm8k", "svamp", 'tabmwp']:
        messages = [HumanMessage(content=f"Use python code solve the following problem, variable <answer> should contain the final answer. Use \"print(answer)\" to get the final answer.\n{item['question']}\n{item["prediction"]}")]
        input = {"react_messages": [messages], "input": f"Use python code solve the following problem, variable <answer> should contain the final answer. Use \"print(answer)\" to get the final answer.\n{item['question']}\n", "prediction": item["prediction"], "iteration": 0}
    try:
        return await asyncio.wait_for(graph.ainvoke(input=input), timeout=timeout)
    except asyncio.TimeoutError:
        print(f"Timeout processing item {item}")
        return f"Timeout on {item}"
    except Exception as e:
        print(f"Error processing item {item}: {e}")
        return f"Error on {item}: {str(e)}"

# 测试 hotpot_qa gsm8k svamp trivia_qa ambig_qa

In [16]:
from datasets import load_dataset
from tqdm.asyncio import tqdm_asyncio

dataset_name = "ambig_qa"
mode = "critic"
num_test_sample = 1000
temperature = 0
top_p = 1
batch_size = 100
llm = ChatOpenAI(temperature=temperature, model="gpt-4o-mini", base_url="https://api.chsdw.top/v1", top_p=top_p)
results = []
dataset = load_dataset("json", data_files="/Users/ariete/Projects/self-improve/output/inference/ambig_qa/cot_1000_temperature_0_top-p_1.jsonl", split="train")
if dataset_name == "svamp":
    dataset = dataset.map(lambda x: {"question": f"{x["Body"]} {x["Question"]}", "answer": x["Answer"]})
print(dataset, dataset[1])
print(llm.temperature, llm.top_p)

Dataset({
    features: ['question', 'answer', 'prediction'],
    num_rows: 1000
}) {'question': 'How often does spermatogeneis—the production of sperm—occur?', 'answer': ['usually continues uninterrupted until death', '74 days', 'constant'], 'prediction': "A: Spermatogenesis occurs continuously throughout a male's life after puberty, with new sperm cells being produced daily. FINAL ANSWER: Continuously after puberty."}
0.0 1.0


In [17]:
from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio
for i in tqdm(range(0, 1000, batch_size)):
    batch = dataset.select(range(i, min(i + batch_size, len(dataset))))
    batch_results = await tqdm_asyncio.gather(*(process(item, dataset_name) for item in batch))
    results.extend(batch_results)
    with open(f"/Users/ariete/Projects/self-improve/output/v2/{dataset_name}/{mode}_{len(dataset)}_temperature_{temperature}_top-p_{top_p}.jsonl", 'w') as f:
        for idx, item in enumerate(results):
            if isinstance(item, str):
                temp = {"idx": idx, "question":  dataset[idx]["question"], "predictions": [dataset[idx]["prediction"]], "answer": dataset[idx]["answer"]}
                f.write(json.dumps(temp) + "\n")
            else:
                temp = {"idx": idx, "question": dataset[idx]["question"], "predictions": item["predictions"], "answer": dataset[idx]["answer"], "prediction": dataset[idx]["prediction"]}
                f.write(json.dumps(temp) + "\n")


  0%|          | 0/10 [00:00<?, ?it/s]WARNING:langchain_core.callbacks.base:AsyncCallbackManager.merge(): Parent run IDs do not match. Using the parent run ID of the first callback manager.
/opt/homebrew/Caskroom/miniconda/base/envs/selfimprove/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/homebrew/Caskroom/miniconda/base/envs/selfimprove/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
 10%|█         | 1/10 [05:00<45:01, 300.17s/it]

Timeout processing item {'question': 'When does episode 37 of stuck in the middle air?', 'answer': ['October 27, 2017'], 'prediction': 'A: Episode 37 of "Stuck in the Middle," titled "Stuck in the Middle of the Road," aired on March 3, 2017. FINAL ANSWER: March 3, 2017.'}
Timeout processing item {'question': 'Who played Marlo Kelly on the drew carey show?', 'answer': ['Jenny McCarthy'], 'prediction': 'A: Marlo Kelly did not appear on "The Drew Carey Show." However, the character Marlo was portrayed by actress Jennifer Aniston in a guest appearance. FINAL ANSWER: Jennifer Aniston (guest appearance).'}


/opt/homebrew/Caskroom/miniconda/base/envs/selfimprove/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/homebrew/Caskroom/miniconda/base/envs/selfimprove/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Timeout processing item {'question': 'When did the metropolitan police force become a service?', 'answer': ['29 September 1829', '1989'], 'prediction': 'A: The Metropolitan Police Force was rebranded as the Metropolitan Police Service in 1829. FINAL ANSWER: 1829'}
Timeout processing item {'question': 'Who has won the 2017 asian snooker championship?', 'answer': ['Lyu Haotian', 'Lyu'], 'prediction': 'A: The 2017 Asian Snooker Championship was won by Pankaj Advani. FINAL ANSWER: Pankaj Advani'}
Timeout processing item {'question': 'Who was the first performance on live aid at John F. Kennedy Stadium, prior to the first musical act?', 'answer': ['Bernard Watson'], 'prediction': 'A: The first performance on the Live Aid stage at John F. Kennedy Stadium was by actor and comedian Jack Nicholson, who introduced the event. The first musical act to perform was "The Hooters." FINAL ANSWER: Jack Nicholson'}
Timeout processing item {'question': 'When did breaking dawn part 2 score come out?', 'ans

 20%|██        | 2/10 [10:00<40:02, 300.31s/it]

...


 90%|█████████ | 9/10 [45:03<05:00, 300.43s/it]WARNING:langchain_core.callbacks.base:AsyncCallbackManager.merge(): Parent run IDs do not match. Using the parent run ID of the first callback manager.
/Users/ariete/Projects/self-improve/agent/utils/tools.py:83: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  result["title"] = BeautifulSoup(result["title"], "html.parser").get_text()
/Users/ariete/Projects/self-improve/agent/utils/tools.py:83: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  result["title"] = BeautifulSoup(result["title"], "html.parser").get_text()


Timeout processing item {'question': 'Who is the 11th game minister of india?', 'answer': ['Vijay Goel'], 'prediction': 'A: The 11th Minister of Youth Affairs and Sports in India is Anurag Thakur, who took office in July 2021. FINAL ANSWER: Anurag Thakur'}
Timeout processing item {'question': 'At which outside locations was It Came From Outer Space filmed at?', 'answer': ['Palmdale, CA, Victorville, CA, and the Mojave Desert'], 'prediction': 'A: "It Came From Outer Space" was primarily filmed at the Mojave Desert in California, specifically near the area of Edwards Air Force Base. Other locations in and around the California desert were also used for various scenes. FINAL ANSWER: Mojave Desert, California.'}
Timeout processing item {'question': 'What is the most games the Red Sox have won between the regular season and the playoffs?', 'answer': ['119'], 'prediction': 'A: The Boston Red Sox won a total of 108 games during the 2018 season, which included both regular season and postseaso

100%|██████████| 100/100 [05:00<00:00,  3.00s/it]


Timeout processing item {'question': 'Kind of oil that has been turned into a solid fat?', 'answer': ['vegetable oil', 'palm oil', 'vegetable oils', 'hydrogenated vegetable oils'], 'prediction': 'A: The kind of oil that has been turned into a solid fat is typically referred to as "hydrogenated oil" or "trans fat." Common examples include margarine and shortening. FINAL ANSWER: Hydrogenated oil.'}
Timeout processing item {'question': 'Who played Zordon in the original power rangers movie?', 'answer': ['Nicholas Bell'], 'prediction': 'A: Zordon was portrayed by actor Nicholas Bell in the original "Mighty Morphin Power Rangers: The Movie" released in 1995. FINAL ANSWER: Nicholas Bell'}
Timeout processing item {'question': 'Where does the peanut butter jelly time popularity come from?', 'answer': ['a Flash-animated music video featuring a dancing banana'], 'prediction': 'A: The "Peanut Butter Jelly Time" meme gained popularity from a flash animation that features a dancing banana and the c

100%|██████████| 10/10 [50:03<00:00, 300.38s/it]


# GSM8K

In [14]:
from datasets import load_dataset
dataset_name = "gsm8k"
mode = "critic"
num_test_sample = -1
temperature = 0
top_p = 1
batch_size = 100
dataset = load_dataset("json", data_files="/Users/ariete/Projects/self-improve/output/inference/gsm8k/pot_1319_temperate_0_top-p_1.jsonl", split="train")
print(dataset)
results = []


Dataset({
    features: ['question', 'answer', 'prediction'],
    num_rows: 1319
})


In [ ]:
from tqdm import tqdm
for i in tqdm(range(100, len(dataset), batch_size)):
    batch = dataset.select(range(i, min(i + batch_size, len(dataset))))
    batch_results = await asyncio.gather(*(process(item, dataset_name) for item in batch))
    results.extend(batch_results)
    with open(f"/Users/ariete/Projects/self-improve/output/v2/{dataset_name}/{mode}_{len(dataset)}_temperature_{temperature}_top-p_{top_p}.jsonl", 'w') as f:
        for idx, item in enumerate(results):
            if isinstance(item, str):
                temp = {"idx": idx, "question":  dataset[idx]["question"], "predictions": [dataset[idx]["prediction"]], "answer": dataset[idx]["answer"]}
                f.write(json.dumps(temp) + "\n")
            else:
                temp = {"idx": idx, "question": dataset[idx]["question"], "predictions": item["predictions"], "answer": dataset[idx]["answer"], "prediction": dataset[idx]["prediction"]}
                f.write(json.dumps(temp) + "\n")